In [ ]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression


nltk.download('punkt')
nltk.download('stopwords')


model = api.load("word2vec-google-news-300")

stop_words = set(stopwords.words('english'))
negation_words = ['not', "don't", 'no', 'never', "can't", "won't"]

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = []
    negation = False
    for word in tokens:
        if word in negation_words:
            negation = not negation
            continue
        if word.isalpha() and word not in stop_words:
            if negation:
                word = 'not_' + word
            filtered_tokens.append(word)
            negation = False
    return filtered_tokens

def get_sentence_vector(tokens):
    vectors = []
    for word in tokens:
        negate = False
        if word.startswith('not_'):
            word = word[4:]
            negate = True
        try:
            vec = model[word]
            if negate:
                vec = -vec
            vectors.append(vec)
        except KeyError:
            continue
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)


data = pd.read_csv("emotions.csv")


numeric_to_string_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

label_mapping = {
    'sadness': 0,
    'joy': 1,
    'love': 2,
    'anger': 3,
    'fear': 4,
    'surprise': 5
}


X = []
y = []

print("Processing text data...")
for idx, row in data.iterrows():
    sentence, label = row['text'], row['label']
    tokens = preprocess_text(sentence)
    vector = get_sentence_vector(tokens)
    X.append(vector)
    y.append(label)


X = np.array(X)
y = np.array(y)


print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Computing mean vectors for each class...")
mean_vectors = []
for cl in np.unique(y_train):
    mean_vectors.append(np.mean(X_train[y_train == cl], axis=0))


print("Computing within-class scatter matrix S_W...")
S_W = np.zeros((model.vector_size, model.vector_size))
for cl, mv in zip(np.unique(y_train), mean_vectors):
    class_scatter = np.zeros((model.vector_size, model.vector_size))
    for row in X_train[y_train == cl]:
        row, mv_col = row.reshape(model.vector_size, 1), mv.reshape(model.vector_size, 1)
        class_scatter += (row - mv_col).dot((row - mv_col).T)
    S_W += class_scatter


print("Computing between-class scatter matrix S_B...")
overall_mean = np.mean(X_train, axis=0).reshape(model.vector_size, 1)
S_B = np.zeros((model.vector_size, model.vector_size))
for i, mean_vec in enumerate(mean_vectors):
    n = X_train[y_train == i, :].shape[0]
    mean_vec = mean_vec.reshape(model.vector_size, 1)
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)


print("Solving the eigenvalue problem...")
eig_vals, eig_vecs = np.linalg.eigh(np.linalg.pinv(S_W).dot(S_B))

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]


eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)


k = len(np.unique(y_train)) - 1
W = np.hstack([eig_pairs[i][1].reshape(model.vector_size, 1) for i in range(k)])


print("Projecting data onto LDA space...")
X_train_lda = X_train.dot(W)
X_test_lda = X_test.dot(W)




print("Training Logistic Regression classifier...")

classifier = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
classifier.fit(X_train_lda, y_train)


print("Predicting on test data...")
y_pred = classifier.predict(X_test_lda)


accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=[numeric_to_string_mapping[i] for i in range(6)]))

